<a href="https://colab.research.google.com/github/dellaanima/DLton_Zero_to_One/blob/main/finetune-hfmodel/stopwords_AUG(A%2CB)_klue_bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install transformers -U

Refenrece : https://wikidocs.net/119990

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#  TPU 초기화
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)



In [13]:
strategy = tf.distribute.TPUStrategy(resolver)

In [14]:
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re


In [15]:
data = pd.read_csv('/content/drive/MyDrive/dlton/data/TRAIN_FINAL_v1.csv')
data.head(20)

,Unnamed: 0,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4342 entries, 0 to 4341
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    4342 non-null   int64 
 1   class         4342 non-null   object
 2   conversation  4342 non-null   object
dtypes: int64(1), object(2)
memory usage: 101.9+ KB


In [17]:
data.duplicated().sum()

0

In [18]:
data['class'].value_counts()

협박 대화          1239
기타 괴롭힘 대화      1094
갈취 대화          1030
직장 내 괴롭힘 대화     979
Name: class, dtype: int64

In [22]:
# 불용어 목록 파일을 읽어 리스트로 변환
with open('/content/stopwords-ko.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f]


In [24]:
stopwords[0:10]

['가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이']

In [25]:
# 불용어 제거 함수 정의
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 데이터프레임의 'text' 열에 불용어 제거 적용
data['conversation_without_stopwords'] = data['conversation'].apply(remove_stopwords)

In [26]:
data

,Unnamed: 0,class,conversation,conversation_without_stopwords
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,지금 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 죽지 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없어....
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,거기 예?? 말이야 너. 이리 오라고 일. 옷 좋아보인다? 얘 돈 있나봐 아니에요....
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,저기요 날이 너무 뜨겁잖아요? 회사에서 선크림 파는데 한 번 손등에 발라보실래요? ...
...,...,...,...,...
4337,4337,갈취 대화,"안녕, 학생!\n 뭐? 나?\n 네, 이리 오세요\n 뭐라고? 왜요. 그런 말이 있...","안녕, 학생! 뭐? 나? 네, 이리 오세요 뭐라고? 왜요. 그런 말이 있어요. 돈 ..."
4338,4338,갈취 대화,"이봐, 이리 오렴. 내게 돈을 좀 주세요.\n나? 나 돈 없어\n내가 맞으면 줄 거...","이봐, 이리 오렴. 내게 돈을 주세요. 나? 돈 없어 내가 맞으면 줄 거야? 내가 ..."
4339,4339,갈취 대화,"이봐.\n어? 나?\n그래, 너.\n왜 그래, 얘야?\n아저씨, 제가 어린아이인가요...","이봐. 어? 나? 그래, 너. 그래, 얘야? 아저씨, 제가 어린아이인가요? 그것보다..."
4340,4340,갈취 대화,내가 학교에 있는 동안 게임 머니를 충전해줘\n 나도 학교에 간다고?\n 게임 머니...,내가 학교에 있는 게임 머니를 충전해줘 나도 학교에 간다고? 게임 머니 충전 얼마나...


In [27]:
def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)
    sentence = sentence.strip()

    return sentence


In [28]:

data['conversation_without_stopwords'] = data['conversation_without_stopwords'].apply(lambda x : preprocess_sentence(x))

data.head(20)


,Unnamed: 0,class,conversation,conversation_without_stopwords
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,지금 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거면 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,거기 예 ? ? 말이야 너 . 이리 오라고 일 . 옷 좋아보인다 ? 얘 돈 있나봐 ...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,저기요 날이 너무 뜨겁잖아요 ? 회사에서 선크림 파는데 한 번 손등에 발라보실래요 ...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....,이틀뒤에 가나다 음식점 예약좀 해줘 . 저녁 시로 . 가나다 음식점이요 . ? 응 ...
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...,번 손님 아이스커피 두잔나왔습니다 아이스커피 ? 맛있게드세요 저기요 아가씨 나는 아...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...,경비아저씨 내차에 경고장 붙였어요 ? 내가 몇년째 사는데 아직도 내차인줄 모르고 이...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...,이거 할인 된다면서요 확인해보겠습니다 아까 시람은 할인 해주던데 손님은 고객님이십니...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데 ? 그게 . 해먹었어 ? 괜찮아...


Label Encoding

In [29]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data['class'])
num_labels = len(label_encoder.classes_)

data['class'] = np.asarray(label_encoder.transform(data['class']), dtype=np.int32)
data.head(10)


,Unnamed: 0,class,conversation,conversation_without_stopwords
0,0,3,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,지금 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거면 ...
1,1,3,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...
2,2,1,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 재미...
3,3,0,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,거기 예 ? ? 말이야 너 . 이리 오라고 일 . 옷 좋아보인다 ? 얘 돈 있나봐 ...
4,4,0,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,저기요 날이 너무 뜨겁잖아요 ? 회사에서 선크림 파는데 한 번 손등에 발라보실래요 ...
5,5,2,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....,이틀뒤에 가나다 음식점 예약좀 해줘 . 저녁 시로 . 가나다 음식점이요 . ? 응 ...
6,6,1,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...,번 손님 아이스커피 두잔나왔습니다 아이스커피 ? 맛있게드세요 저기요 아가씨 나는 아...
7,7,1,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...,경비아저씨 내차에 경고장 붙였어요 ? 내가 몇년째 사는데 아직도 내차인줄 모르고 이...
8,8,1,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...,이거 할인 된다면서요 확인해보겠습니다 아까 시람은 할인 해주던데 손님은 고객님이십니...
9,9,0,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데 ? 그게 . 해먹었어 ? 괜찮아...


In [30]:
data['class'].value_counts()

3    1239
1    1094
0    1030
2     979
Name: class, dtype: int64

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# conversation : x , label : y
conversation = data['conversation_without_stopwords'].tolist()
label = data['class'].tolist()

# train, val 분리 (데이터 0.1)
train_x, val_x, train_y, val_y = train_test_split(conversation, label, random_state=42, test_size=0.1)

In [34]:
len(train_x), len(val_x)

(3907, 435)

In [35]:
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM

HUGGINGFACE_MODEL_PATH = "klue/bert-base"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
# Tokenizing
train_encodings = tokenizer(train_x, truncation=True, padding=True)
val_encodings = tokenizer(val_x, truncation=True, padding=True)

In [36]:
# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_y
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_y
))

In [37]:
num_labels

4

In [38]:
from transformers import TFBertForSequenceClassification


with strategy.scope():
  num_labels = num_labels
  model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3076      
                                                                 
Total params: 110,620,420
Trainable params: 110,620,420
Non-trainable params: 0
_________________________________________________________________


In [40]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=3)

history = model.fit(
    train_dataset.shuffle(len(train_dataset)).batch(16), epochs=3, batch_size=16,
    validation_data=val_dataset.shuffle(len(val_dataset)).batch(16),
    callbacks = [callback_earlystop]
)


Epoch 1/3
245/245 [==============================] - 161s 156ms/step - loss: 0.4426 - accuracy: 0.8480 - val_loss: 0.3242 - val_accuracy: 0.8966
Epoch 2/3
245/245 [==============================] - 28s 115ms/step - loss: 0.1727 - accuracy: 0.9473 - val_loss: 0.2561 - val_accuracy: 0.9287
Epoch 3/3
245/245 [==============================] - 29s 118ms/step - loss: 0.0813 - accuracy: 0.9754 - val_loss: 0.4556 - val_accuracy: 0.8644


In [41]:


id2labels = model.config.id2label
model.config.id2label = {id : label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0]  for id, label in id2labels.items()}

label2ids = model.config.label2id
model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] : id   for id, label in id2labels.items()}



In [42]:
import os
DIR_PATH = '/content/drive/MyDrive/dlton/model'
MODEL_NAME = 'stop_AUG(A,B)_klue_bert_finetune'
MODEL_SAVE_PATH = os.path.join(DIR_PATH, MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune -- Folder create complete 



('/content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune/tokenizer_config.json',
 '/content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune/special_tokens_map.json',
 '/content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune/vocab.txt',
 '/content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune/added_tokens.json',
 '/content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune/tokenizer.json')

In [43]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizer.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    top_k=None
)

Some layers from the model checkpoint at /content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/dlton/model/stop_AUG(A,B)_klue_bert_finetune.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions w

In [44]:


# Load Test-set
test_file_path = '/content/drive/MyDrive/dlton/data/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)
test_dataset



,t_000,t_001,t_002,t_004,t_005,t_006,t_007,t_009,t_010,t_012,...,t_489,t_490,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
text,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...,그 사람 누구냐? 누구말하는거야? 다 알면서 모른척 하지마. 둘다 쏴버리기 전에...,야 저기야 닭꼬치 트럭왔다 응 그러네 그치? 너도 먹고 싶지? 어?나도? 그래 너 ...,야 너 2학년 김민석 맞지? 네 맞는데요. 혹시 누구신가요? 내가 누군지 궁금하면 ...,예전에 네가 나한테 했던 일 기억하지? 너도 당해봐 예전 일이라면 내가 사과할게. ...,김비서 지금 우리애 학교가서 집에다가 좀 데려다 주세요 사장님 그런 개인적인 일은 ...,...,너 어제 집에 바로 안 들어갔지? 어떻게 알았어? 내가 눈이 좋아 왜 묻는건데? 너...,철수야 말 들어야지 싫은데 철밥통 선생아 너 선생님한테 말버릇이 뭐야 야 내가 너말...,동생이쁘다. 나주라 안돼 우리동생건들지마. 왜 안되냐? 좀 달라고 니 목숨값으로 동...,혹시 나 오늘 2만원만 빌려주만 안돼? 응? 안돼. 나도 요즘 맨날 택시타느라 택시...,이봐 네? 누구세요? 입다물고 가진거 다내놔 경찰에 신고할거에요 할 수 있으면 해봐...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,엄마 저 그 돈 안해주시면 정말 큰일나요. 이유도 말하지 않고. 몇번째니 경민아....


In [47]:
test_data = test_dataset.transpose()

In [48]:
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [49]:
# 데이터프레임의 'text' 열에 불용어 제거 적용
test_data['text'] = test_data['text'].apply(remove_stopwords)
test_data['text'] = test_data['text'].apply(lambda x : preprocess_sentence(x))



In [50]:
test_data

,text
t_000,아가씨 담배한갑주소 원입니다 지갑어디갔지 에이 버스에서 잃어버렸나보네 취소할까요 아...
t_001,우리팀에서 다른팀으로 갈 사람 없나 ? 영지씨가 가는건 어때 ? 네 ? 제가요 ? ...
t_002,오늘 그게 뭐야 제가 뭘 잘못했나요 . ? 제대로 하지 똑바로 하지 행실이 맘에 안...
t_004,아무튼 앞으로 니가 내 와이파이야 . . 응 와이파이 온 . 켰어 . 반말 ? 주인...
t_005,빨리 말해 . 선생님 제발 살려주십시오 . 비밀번호 틀릴 때마다 손톱 하나씩 뺀찌로...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 ? 합니다 . 보고서...
t_496,교수님 논문에 이름이 없나요 ? 논문말이야 ? 지난 번 냈던 논문이요 . 그거 제가...
t_497,저요 ? 왜요 돈좀 줘봐 돈 없어요 돈이 없어 지갑은 폼이니 ? 진짜 없어요 줘봐 ...
t_498,빨리 안 뛰어와 ? 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가 지금 과장...


In [ ]:
predicted_label_list = []

for text in test_data['text']:
    # predict
    preds_list = text_classifier(text)[0][0]
    predicted_label_list.append(preds_list['label']) # label

In [ ]:
submission = test_data.assign(CLASS=predicted_label_list)
submission.rename_axis('file_name', inplace=True)
submission = submission.rename(columns={'CLASS':'class'})

In [ ]:
submission.drop(['text'], axis=1, inplace=True)

In [ ]:

class_mapping = {
'협박 대화': 0,
'갈취 대화': 1,
'직장 내 괴롭힘 대화': 2,
'기타 괴롭힘 대화': 3

}
submission['class'] = [class_mapping[label] for label in submission['class']]
submission['class'] = np.asarray(submission['class'], dtype=np.int32)

In [ ]:
submission.head(10)

In [ ]:
import pandas as pd


submission_file_path = '/content/drive/MyDrive/dlton/stop_AUG_AB_klue-bert_3epo_submission.csv'
submission.to_csv(submission_file_path, index=True)

